In [1]:
# Required imports:
import pandas as pd
import plotly.express as px
import numpy as np

# Eurostat to Map data
This next block of code transforms the volume of passengers relative to GDP data
to a dataframe which can be used in a choropleth map.
Parts of this can be reused for other eurostat data sets.

In [37]:
# Loads in European countries and their iso-alpha-3 codes as a dictionary
df_alpha = pd.read_csv("Data/countries_iso.csv")
dic_alpha = df_alpha.set_index("country").T.to_dict("list")

# Transposes data and removes empty/unnecessary columns
df = pd.read_csv("Data/pg_GDP.csv")
df.set_index(df.columns[0], inplace=True)
df = df.T
remove_rows = np.r_[np.r_[0:6, 7, 9:11, 49:57]]
df.drop(df.columns[remove_rows], axis=1, inplace=True)

# Rename some countries for consistency with the iso-alpha-3 dictionary
df.rename(columns={"European Union - 27 countries (from 2020)": "EU", "Czechia": "Czech Republic", "Türkiye": "Turkey"}, inplace=True)

# Create a new dataframe usable by choropleth using the transposed data
df_processed = pd.DataFrame(columns=["country", "iso_alpha", "year", "pg_GDP", "flag"])
for row in range(0, len(df.index), 2):
    for col in range(1, len(df.columns)):
        if df.columns[col] == "EU":
            iso_alpha = None
        else:
            iso_alpha = dic_alpha[df.columns[col]][0]
        df_processed.loc[len(df_processed)] = [df.columns[col],
                                                             iso_alpha,
                                                             df.iloc[row, 0],
                                                             df.iloc[row, col],
                                                             df.iloc[row+1, col]]

# Replace NaN with 0 and save dataframe
df_processed = df_processed.replace(":", 0)
df_processed.to_csv("Data/pg_GDP_map.csv")

# Choropleth Map using Plotly
This next block of code creates a map of Europe using plotly's choropleth.
In this particular instance, it plots transport volumes relative to GDP.
Most of this code can be reused for plotting other data sets using choropleth.

In [14]:
# Read in prepared data set, rename a column and extract the different years
df = pd.read_csv("Data/pg_GDP_map.csv")
df = df.rename(columns={"pg_GDP": "index"})
years = df.year.unique()

# Calculate max value for continuous colour range
max_pg_gdp = df['index'].max()

# Get year from user and apply to dataset
given_year = input("Give a year between 2010 and 2021: ")
df_new = df.query('year=='+str(given_year))
customdata = df_new['country']

# Create choropleth figure
fig = px.choropleth(df_new, 
                    locations="iso_alpha",  # Iso-alpha-3 codes to signify which country
                    color="index",
                    hover_name="country",
                    color_continuous_scale=[[0, 'gray'], [0.01, 'gray'], [0.01, 'blue'], [1, 'red']],  # Gray represents countries with no available data.
                    projection='miller',
                    range_color=(0,max_pg_gdp),
                    scope='europe'
                    )

# Sets hover data
fig.update_traces(customdata=customdata, 
                  hovertemplate= np.select([df_new["index"] == 0], ["<b>Country: </b> %{customdata}<br><br>No data available"], "<b>Country: </b>%{customdata}<br><br><b>Index value: </b>%{z}"))

# Gets value for the entire EU
numb = df_new.query('country=="EU"')["index"]

# Set title and EU index and legend position
fig.update_layout(title="<b>Volume of passengers relative to GDP in " + given_year + " (2015 base year)</b><br>EU index: " + str(numb.tolist()[0]),
                   coloraxis_colorbar=dict(x=0,
                                           y=0.5)
                  )

# Set figure size
fig.update_layout(width=1000, 
                  height=600)

fig.show()